In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import mixture
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm, linear_model, datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import (confusion_matrix, precision_score, recall_score,
                             accuracy_score, roc_auc_score, RocCurveDisplay)
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from tabulate import tabulate
from sklearn.metrics import mean_squared_error
df = pd.read_csv("Merged_WB_PWT_DEMO.csv", encoding="utf-8-sig")
df.columns = [c.lower().strip().replace(" ", "_") for c in df.columns]
df = df.replace('..', np.nan)
wb_wide = df.pivot_table(
    index=["country", "year"],
    columns="series_code",
    values="wb_value",
    aggfunc="first"
).reset_index()
wb_rename = {
    "NY.GDP.PCAP.KD": "gdp_pc",             
    "NE.TRD.GNFS.ZS": "trade_openness",     
    "FP.CPI.TOTL.ZG": "inflation",         
    "BX.KLT.DINV.WD.GD.ZS": "FDI",         
    "NE.GDI.TOTL.ZS": "investment_ratio"    
}
wb_wide = wb_wide.rename(columns={k: v for k, v in wb_rename.items() if k in wb_wide.columns})
if "gdp_pc" in wb_wide.columns:
    wb_wide["gdp_pc"] = pd.to_numeric(wb_wide["gdp_pc"], errors="coerce")
    wb_wide = wb_wide.sort_values(["country", "year"])
    wb_wide["gdp_pc_growth"] = (
        wb_wide.groupby("country")["gdp_pc"].pct_change() * 100
    )
base_cols = [
    "country", "country_code", "series_name", "series_code",
    "year", "wb_value", "countrycode", "currency_unit",
    "demographic_indicator", "unnamed:_3", "unnamed:_4",
    "unnamed:_5", "unnamed:_6"
]
base_cols = [c for c in base_cols if c in df.columns]
pwt_cols = [c for c in df.columns if c not in base_cols]
pwt_panel = df.groupby(["country", "year"], as_index=False)[pwt_cols].first()
if "pop" in pwt_panel.columns:
    pwt_panel = pwt_panel.sort_values(["country", "year"])
    pwt_panel["pop_growth"] = (
        pwt_panel.groupby("country")["pop"].pct_change() * 100
    )
if "hc" in pwt_panel.columns:
    pwt_panel = pwt_panel.rename(columns={"hc": "education"})
wgi = pd.read_excel("wgidataset.xlsx")
wgi.columns = [c.lower().strip() for c in wgi.columns]
wgi = wgi.rename(columns={"countryname": "country"})
wgi["estimate"] = pd.to_numeric(wgi["estimate"], errors="coerce")
wgi_pivot = wgi.pivot_table(
    index=["country", "year"],
    columns="indicator",
    values="estimate",
    aggfunc="mean"
).reset_index()
print("WGI columns:", wgi_pivot.columns)
panel = wb_wide.merge(pwt_panel, on=["country", "year"], how="left") \
               .merge(wgi_pivot, on=["country", "year"], how="left")
print(panel.columns.tolist())
print(panel.head())

/var/folders/92/394r745n3cxg7hwft9b6jmt40000gn/T/ipykernel_10763/3393062247.py:19: DtypeWarning: Columns (56,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Merged_WB_PWT_DEMO.csv", encoding="utf-8-sig")


WGI columns: Index(['country', 'year', 'cc', 'ge', 'pv', 'rl', 'rq', 'va'], dtype='object', name='indicator')
['country', 'year', 'EG.ELC.ACCS.ZS', 'GC.DOD.TOTL.GD.ZS', 'GC.NLD.TOTL.GD.ZS', 'MS.MIL.XPND.GD.ZS', 'trade_openness', 'NY.ADJ.NNTY.KD.ZG', 'NY.GDP.MKTP.KD.ZG', 'gdp_pc', 'gdp_pc_growth', 'rgdpe', 'rgdpo', 'pop', 'emp', 'avh', 'education', 'ccon', 'cda', 'cgdpe', 'cgdpo', 'cn', 'ck', 'ctfp', 'cwtfp', 'rgdpna', 'rconna', 'rdana', 'rnna', 'rkna', 'rtfpna', 'rwtfpna', 'labsh', 'irr', 'delta', 'xr', 'pl_con', 'pl_da', 'pl_gdpo', 'i_cig', 'i_xm', 'i_xr', 'i_outlier', 'i_irr', 'cor_exp', 'csh_c', 'csh_i', 'csh_g', 'csh_x', 'csh_m', 'csh_r', 'pl_c', 'pl_i', 'pl_g', 'pl_x', 'pl_m', 'pl_n', 'pl_k', 'pop_growth', 'cc', 'ge', 'pv', 'rl', 'rq', 'va']
   country  year EG.ELC.ACCS.ZS GC.DOD.TOTL.GD.ZS GC.NLD.TOTL.GD.ZS  \
0  Albania  1980            NaN               NaN               NaN   
1  Albania  1981            NaN               NaN               NaN   
2  Albania  1982            Na

In [2]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
target = "gdp_pc_growth"
candidate_features = [
    "va", "pv", "ge", "rq", "rl", "cc",
    "pop_growth", "education",
    "trade_openness", "inflation", "FDI", "investment_ratio",
    "rgdpe", "rgdpo", "pop"
]
features = [f for f in candidate_features if f in panel.columns]
#print("Using features:", features)
model_df = panel.dropna(subset=[target]).copy()
model_df = model_df.replace('..', np.nan)
for col in features + [target]:
    model_df[col] = pd.to_numeric(model_df[col], errors='coerce')
model_df = model_df.dropna(subset=features + [target])
X = model_df[features]
y = model_df[target]
print(X.dtypes)

print("Any '..' left in X? ->", (X == '..').any().any())
y_binned = pd.qcut(y, q=3, labels=['Low', 'Medium', 'High'])
y_numeric = y_binned.map({'Low': 0, 'Medium': 1, 'High': 2})
#turns our y variable into an integer value for low, medium, and high
#goal is to have a classification model

corr_matrix = X.corr().abs()

# Upper triangle to avoid duplicate pairs
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find columns with correlation > 0.85
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
print("Highly correlated features to consider dropping:", to_drop)

# Drop them from X
X_reduced = X.drop(columns=to_drop)

X_train, X_temp, y_train, y_temp = train_test_split(
    X_reduced, y_numeric, test_size=0.25, random_state=42
)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=.5,random_state=0)

va                float64
pv                float64
ge                float64
rq                float64
rl                float64
cc                float64
pop_growth        float64
education         float64
trade_openness    float64
rgdpe             float64
rgdpo             float64
pop               float64
dtype: object
Any '..' left in X? -> False
Highly correlated features to consider dropping: ['rq', 'rl', 'cc', 'rgdpo']


In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
param_grid = {
    'n_estimators': [300,600,900],
    'max_depth': [6,10,15],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1, 2, 4]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV score:", grid.best_score_)

Best params: {'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}
Best CV score: 0.5306850714658209


In [4]:


#rf = RandomForestClassifier(max_depth = 12, min_samples_leaf =1, min_samples_split = 2, n_estimators = 200, random_state = 42)
#rf.fit(X_train, y_train)
y_pred_valid = grid.predict(X_valid)
print("Validation Accuracy:", accuracy_score(y_valid, y_pred_valid))
print("\nClassification Report:\n", classification_report(y_valid, y_pred_valid))
print("\nConfusion Matrix:\n", confusion_matrix(y_valid, y_pred_valid))

# Optional: evaluate on test set
y_pred_test = grid.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred_test))
print(X_reduced.columns)


Validation Accuracy: 0.590027700831025

Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.54      0.58       134
           1       0.46      0.56      0.51       108
           2       0.71      0.66      0.68       119

    accuracy                           0.59       361
   macro avg       0.60      0.59      0.59       361
weighted avg       0.60      0.59      0.59       361


Confusion Matrix:
 [[73 47 14]
 [28 61 19]
 [16 24 79]]
Test Accuracy: 0.5359116022099447
Index(['va', 'pv', 'ge', 'pop_growth', 'education', 'trade_openness', 'rgdpe',
       'pop'],
      dtype='object')
